In [1]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
import laspy


import time
import requests
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [11]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_username = 'spiros.koulouzis@lifewatch.eu'
if 'JUPYTERHUB_USER' in os.environ:
    conf_username = os.environ['JUPYTERHUB_USER']
    
param_remote_path_root = '/webdav/vl-laserfarm/ahn'
conf_remote_path_split = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/split')
conf_remote_path_retiled = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/retiled')
conf_remote_path_norm = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/norm')
conf_remote_path_targets = pathlib.Path('/webdav/vl-laserfarm/' + conf_username +  '/targets')
conf_remote_path_geotiffs = pathlib.Path('/webdav/vl-laserfarm/' + conf_username + '/geotiffs')
conf_local_tmp = pathlib.Path('/tmp/')


param_hostname = ''
param_login = ''
param_password = ''

conf_feature_name = 'perc_95_normalized_height'
conf_validate_precision = '0.001'
conf_tile_mesh_size = '10.'
conf_filter_type= 'select_equal'
conf_attribute = 'raw_classification'
conf_min_x = '-113107.81'
conf_max_x = '398892.19'
conf_min_y = '214783.87'
conf_max_y = '726783.87'
conf_n_tiles_side = '512'
conf_apply_filter_value = '1'
conf_laz_compression_factor = '7'
param_max_filesize = '60'  # desired max file size (in bytes)


conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}

## Fetching Laz Files from remote WebDAV

In [12]:
# S1 Fetch Laz Files
print(param_remote_path_root)
laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), pathlib.Path(param_remote_path_root).as_posix())
             if f.lower().endswith('.laz')]
print(laz_files)

/webdav/vl-laserfarm/ahn
['C_36FN2.LAZ', 'C_64EN2.LAZ']


## Splitting big files into smaller files before retiling
This step can be added if the original files are too large for normal VMs to process

In [13]:
# S2 split big files

import numpy as np

def save_chunk_to_laz_file(in_filename, 
                           out_filename, 
                           offset, 
                           n_points):
    """Read points from a LAS/LAZ file and write them to a new file."""
    points = np.array([])
    
    with laspy.open(in_filename) as in_file:
        with laspy.open(out_filename, 
                        mode="w", 
                        header=in_file.header) as out_file:
            in_file.seek(offset)
            points = in_file.read_points(n_points)
            out_file.write_points(points)
    return out_filename

def split_strategy(filename, max_filesize):
    """Set up splitting strategy for a LAS/LAZ file."""
    with laspy.open(filename) as f:
        bytes_per_point = (
            f.header.point_format.num_standard_bytes +
            f.header.point_format.num_extra_bytes
        )
        n_points = f.header.point_count
    n_points_target = int(
        max_filesize * int(conf_laz_compression_factor) / bytes_per_point
    )
    stem, ext = os.path.splitext(filename)
    return [
        (filename, f"{stem}-{n}{ext}", offset, n_points_target)
        for n, offset in enumerate(range(0, n_points, n_points_target))
    ]

from webdav3.client import Client

client = Client(conf_wd_opts)
client.mkdir(conf_remote_path_split.as_posix())

remote_path_split = str(conf_remote_path_split)

for file in laz_files:
    print('Splitting: '+file )
    client.download_sync(remote_path=os.path.join(param_remote_path_root,file), local_path=file)
    inps = split_strategy(file, int(param_max_filesize)*1048576)
    
    for inp in inps:
        out_filename = save_chunk_to_laz_file(*inp)
        print('out_filename: '+out_filename)
        client.upload_sync(remote_path=os.path.join(conf_remote_path_split,out_filename), local_path=out_filename)
    


Splitting: C_36FN2.LAZ
out_filename: C_36FN2-0.LAZ
Splitting: C_64EN2.LAZ
out_filename: C_64EN2-0.LAZ


In [14]:
# S21 Fetch Laz Files
remote_path_split
split_laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), pathlib.Path(str(conf_remote_path_split)).as_posix())
             if f.lower().endswith('.laz')]
print(split_laz_files)

['C_64EN2-0.LAZ', 'C_36FN2-0.LAZ']


## Retiling of big files into smaller tiles

In [15]:
# S3 Retiling
split_laz_files
remote_path_retiled = str(conf_remote_path_retiled)

print('------------------')
print(split_laz_files)
print('------------------')

grid_retile = {
    'min_x': float(conf_min_x),
    'max_x': float(conf_max_x),
    'min_y': float(conf_min_y),
    'max_y': float(conf_max_y),
    'n_tiles_side': int(conf_n_tiles_side)
}

retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_split.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

for file in split_laz_files:
    clean_file = file.replace('"','').replace('[','').replace(']','')
    print(clean_file)
    retiler = Retiler(clean_file,label=clean_file).config(retiling_input).setup_webdav_client(conf_wd_opts)
    retiler_output = retiler.run()

2023-12-20 21:10:06,877 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/C_64EN2-0.LAZ_input
2023-12-20 21:10:06,880 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/C_64EN2-0.LAZ_output
2023-12-20 21:10:06,885 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/vl-laserfarm/spiros.koulouzis@lifewatch.eu/split/C_64EN2-0.LAZ ...


------------------
['C_64EN2-0.LAZ', 'C_36FN2-0.LAZ']
------------------
C_64EN2-0.LAZ


2023-12-20 21:10:09,000 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2023-12-20 21:10:09,001 -                        laserfarm.retiler -       INFO - Setting up the target grid
2023-12-20 21:10:09,002 -                        laserfarm.retiler -       INFO - Splitting file /tmp/C_64EN2-0.LAZ_input/C_64EN2-0.LAZ with PDAL ...
2023-12-20 21:10:31,460 -                        laserfarm.retiler -       INFO - ... splitting completed.
2023-12-20 21:10:31,465 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2023-12-20 21:10:31,470 -                        laserfarm.retiler -       INFO - ... file C_64EN2-0_1.LAZ to tile_162_210
2023-12-20 21:10:31,473 -                        laserfarm.retiler -       INFO - ... file C_64EN2-0_2.LAZ to tile_163_210
2023-12-20 21:10:31,477 -                        laserfarm.retiler -       INFO - ... file C_64EN2-0_4.LAZ to tile_163_211
2023-12-20 21:10:31,479 -                   

C_36FN2-0.LAZ


2023-12-20 21:10:44,811 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2023-12-20 21:10:44,813 -                        laserfarm.retiler -       INFO - Setting up the target grid
2023-12-20 21:10:44,815 -                        laserfarm.retiler -       INFO - Splitting file /tmp/C_36FN2-0.LAZ_input/C_36FN2-0.LAZ with PDAL ...
2023-12-20 21:11:11,643 -                        laserfarm.retiler -       INFO - ... splitting completed.
2023-12-20 21:11:11,651 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2023-12-20 21:11:11,655 -                        laserfarm.retiler -       INFO - ... file C_36FN2-0_4.LAZ to tile_171_229
2023-12-20 21:11:11,658 -                        laserfarm.retiler -       INFO - ... file C_36FN2-0_2.LAZ to tile_171_228
2023-12-20 21:11:11,662 -                        laserfarm.retiler -       INFO - ... file C_36FN2-0_9.LAZ to tile_173_230
2023-12-20 21:11:11,665 -                   

In [16]:
# S4 Retrive retiled tiles
remote_path_retiled

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

In [ ]:
# S5 Feature Extraction
    
for t in tiles:
    features = [conf_feature_name]

    tile_mesh_size = float(conf_tile_mesh_size)

    grid_feature = {
        'min_x': float(conf_min_x),
        'max_x': float(conf_max_x),
        'min_y': float(conf_min_y),
        'max_y': float(conf_max_y),
        'n_tiles_side': int(conf_n_tiles_side)
    }

    feature_extraction_input = {
        'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
        'pullremote': conf_remote_path_retiled.as_posix(),
        'load': {'attributes': [conf_attribute]},
        'normalize': 1,
        'apply_filter': {
            'filter_type': conf_filter_type, 
            'attribute': conf_attribute,
            'value': [int(conf_apply_filter_value)]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
        },
        'generate_targets': {
            'tile_mesh_size' : tile_mesh_size,
            'validate' : True,
            'validate_precision': float(conf_validate_precision),
            **grid_feature
        },
        'extract_features': {
            'feature_names': features,
            'volume_type': 'cell',
            'volume_size': tile_mesh_size
        },
        'export_targets': {
            'attributes': features,
            'multi_band_files': False
        },
        'pushremote': conf_remote_path_targets.as_posix(),
    #     'cleanlocalfs': {}
    }
    idx = (t.split('_')[1:])

    processing = DataProcessing(t, tile_index=idx,label=t).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
    processing.run()
    
remote_path_targets = conf_remote_path_targets.as_posix()

print(type(remote_path_targets))

2023-12-20 21:11:34,063 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/tile_163_211_input
2023-12-20 21:11:34,066 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/tile_163_211_output
2023-12-20 21:11:34,072 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/vl-laserfarm/spiros.koulouzis@lifewatch.eu/retiled/tile_163_211 ...
2023-12-20 21:11:36,451 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2023-12-20 21:11:36,453 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2023-12-20 21:11:36,455 -                laserfarm.data_processing -       INFO - ... loading /tmp/tile_163_211_input/tile_163_211/C_64EN2-0_4.LAZ
2023-12-20 21:11:37,765 -                laserfarm.data_processing -       INFO - ... loading completed.
2023-12-20 21:11:37,768 -                laserfarm.data_processing -       INFO - Normalizing point-cloud 

In [ ]:
# S6 GeoTIFF Export

remote_path_targets

#feature = features

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': conf_remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=conf_feature_name, bands=conf_feature_name, label=conf_feature_name).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()